In [77]:
# LZW Encoder
# Adaptive Arithmetic 

import sys
from sys import argv
from struct import *
import contextlib
import arithmeticcoding
import json

#Creating the input file
idFile= open("input.txt","w")
numberFor = ""
forFile = ""
for i in range(1,10):
    x = 209332253*i
    numberFor = numberFor + str(x)
for i in range(1,101):
    forFile = forFile + numberFor
idFile.write(forFile)


# taking the input file and the number of bits from command line
# defining the maximum table size
# opening the input file
# reading the input file and storing the file data into data variable

n = 8600*8
maximum_table_size = pow(2,int(n))      
file = open("input.txt", "r")                  
data = file.read()                      

# Building and initializing the dictionary.
dictionary_size = 256                   
dictionary = {chr(i): i for i in range(dictionary_size)}    
string = ""             # String is null.
compressed_data = []    # variable to store the compressed data.


# iterating through the input symbols.
# LZW Compression algorithm
for symbol in data:                     
    string_plus_symbol = string + symbol # get input symbol.
    if string_plus_symbol in dictionary: 
        string = string_plus_symbol
    else:
        compressed_data.append(dictionary[string])
        if(len(dictionary) <= maximum_table_size):
            dictionary[string_plus_symbol] = dictionary_size
            dictionary_size += 1
        string = symbol

if string in dictionary:
    compressed_data.append(dictionary[string])

# storing the compressed string into a file (byte-wise).
output_file = open("compressed1.lzw", "wb")
for data in compressed_data:
    output_file.write(pack('>H',int(data)))

#print(type(dictionary))

#print(compressed_data)    
intermediate = open("inter.txt", "w")
intermediate.write("DICTIONARY:\n\n")
intermediate.write(json.dumps(dictionary))
intermediate.write("\n\nINDEXES:\n\n")  
for element in compressed_data:
    intermediate.write(str(element)+ " ")
    
intermediate.close()

output_file.close()
file.close()

forBinaryStrting = ""
# Perform file compression
with open("compressed1.lzw", "rb") as inp, \
        contextlib.closing(arithmeticcoding.BitOutputStream(open("compressed.bin", "wb"))) as bitout:
    initfreqs = arithmeticcoding.FlatFrequencyTable(257)
    freqs = arithmeticcoding.SimpleFrequencyTable(initfreqs)
    enc = arithmeticcoding.ArithmeticEncoder(32, bitout)
    while True:
        # Read and encode one byte
        symbol = inp.read(1)
        if len(symbol) == 0:
            break
        enc.write(freqs, symbol[0])
        freqs.increment(symbol[0])
    enc.write(freqs, 256)  # EOF
    enc.finish()  # Flush remaining code bits 
    
with open("compressed.bin",encoding = "ANSI") as f:
    contents = f.read()

a_byte_array = bytearray(contents, "utf8")
#Create bytearray
byte_list = []
for byte in a_byte_array:
    binary_representation = bin(byte)
#Convert to binary

    byte_list.append(binary_representation)
#Add to list
#print(byte_list)
with open("compressed.txt","w") as f:
   # for data in byte_list
    f.write(str(byte_list))
    

    
    
    

In [78]:
# LZW Decoder
# Adaptive Arithmetic 

import sys
import arithmeticcoding
from sys import argv
import struct
from struct import *


# Perform file decompression
inp = open("compressed.bin", "rb")
out = open("decompressed1.lzw", "wb")
bitin = arithmeticcoding.BitInputStream(inp)
    
initfreqs = arithmeticcoding.FlatFrequencyTable(257)
freqs = arithmeticcoding.SimpleFrequencyTable(initfreqs)
dec = arithmeticcoding.ArithmeticDecoder(32, bitin)
while True:
    # Decode and write one byte
    symbol = dec.read(freqs)
    if symbol == 256:  # EOF symbol
        break
    out.write(bytes((symbol,)))
    freqs.increment(symbol)

inp.close()
out.close()
    

# defining the maximum table size
# opening the compressed file
# defining variables

n = 8600*8
maximum_table_size = pow(2,int(n))
file = open("decompressed1.lzw", "rb")
compressed_data = []
next_code = 256
decompressed_data = ""
string = ""

# Reading the compressed file.
while True:
    rec = file.read(2)
    if len(rec) != 2:
        break
    (data, ) = unpack('>H', rec)
    compressed_data.append(data)

# Building and initializing the dictionary.
dictionary_size = 256
dictionary = dict([(x, chr(x)) for x in range(dictionary_size)])

# iterating through the codes.
# LZW Decompression algorithm
for code in compressed_data:
    if not (code in dictionary):
        dictionary[code] = string + (string[0])
    decompressed_data += dictionary[code]
    if not(len(string) == 0):
        dictionary[next_code] = string + (dictionary[code][0])
        next_code += 1
    string = dictionary[code]

# storing the decompressed string into a file.

output_file = open("output.txt", "w")
for data in decompressed_data:
    output_file.write(data)
    
output_file.close()
file.close()

